<a href="https://colab.research.google.com/github/MatteoGuglielmi-tech/Polarity-and-Subjectivity-Detection/blob/main/src/MyModel/pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2961473020452296410
 xla_global_id: -1]

In [10]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes

In [11]:
!cat /proc/meminfo

MemTotal:       13297228 kB
MemFree:         9944492 kB
MemAvailable:   11804952 kB
Buffers:           62696 kB
Cached:          1884140 kB
SwapCached:            0 kB
Active:           516452 kB
Inactive:        2641904 kB
Active(anon):        968 kB
Inactive(anon):  1100592 kB
Active(file):     515484 kB
Inactive(file):  1541312 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               188 kB
Writeback:             0 kB
AnonPages:       1211672 kB
Mapped:           484416 kB
Shmem:              1260 kB
KReclaimable:      90232 kB
Slab:             121188 kB
SReclaimable:      90232 kB
SUnreclaim:        30956 kB
KernelStack:        4608 kB
PageTables:        21484 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6648612 kB
Committed_AS:    3430092 kB
VmallocTotal:   34359738367 kB
VmallocUsed:        9444 kB
VmallocChunk:          0 kB
Percpu:          

# MTL and BERT Embedding

In [31]:
from nltk.sentiment.util import mark_negation
from typing import List, Dict

def negative_marking(doc : List[str]) -> List[str]:
    '''
        Params :
        -----------------
            doc : list[str]
                document where each element is a list of strings
        Returns :
            negated_doc : list[str]
                document after having applied double negation
    '''

    flat_doc = [w for sent in doc for w in sent]
    negated_doc = mark_negation(flat_doc, double_neg_flip=True)

    return " ".join([w for w in negated_doc])

In [32]:
from textblob.en import subjectivity
import nltk
from nltk.corpus import movie_reviews
from nltk.corpus import subjectivity

nltk.download('punkt')
nltk.download('movie_reviews')
nltk.download('subjectivity')


mr = movie_reviews
sub = subjectivity
neg = mr.paras(categories='neg')
pos = mr.paras(categories='pos')


subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')]

print(len(neg), len(pos))
print(len(subj_docs), len(obj_docs))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package subjectivity to /root/nltk_data...
[nltk_data]   Package subjectivity is already up-to-date!


1000 1000
5000 5000


In [33]:
new_corpus = [negative_marking(d) for d in pos] + [negative_marking(d) for d in neg]

In [34]:
import pandas as pd

data = pd.DataFrame(new_corpus, columns=['text'])

In [35]:
data

,text
0,films adapted from comic books have had plenty...
1,every now and then a movie comes along from a ...
2,you ' ve got mail works alot better than it de...
3,""" jaws "" is a rare film that grabs your attent..."
4,moviemaking is a lot like being the general ma...
...,...
1995,"if anything , "" stigmata "" should be taken as ..."
1996,"john boorman ' s "" zardoz "" is a goofy cinemat..."
1997,the kids in the hall are an acquired taste . i...
1998,there was a time when john carpenter was a gre...


In [36]:
from nltk.corpus import stopwords
import re
import string
from textblob import TextBlob as tb



nltk.download('stopwords')

## https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
# since re are largely used for this type of applications, the regex module is used
# re.sub(pattern, repl, string, count=0, flags=0)

def pre_processing(text : str) -> str :
  '''Clear text from numbers, stop words (very common words), punctuation and 
    correct possible misspelled words

    Params :
    --------
      cw : List[str]
        list of words in the sentence to be cleaned
    Returns :
    ---------
      list of cleaned words
  '''

  stop_list = set(stopwords.words("english"))
  text = list(text.lower().split())
  text = ' '.join([word for word in text if word not in stop_list])
  # remove http links
  text = re.sub(r'http\S+', '', text)
  # Remove hashtags
  text = re.sub(r'#\w*', '', text)
  # Remove whitespace (including new line characters)
  text = re.sub(r'\s\s+', '', text)
  # Remove single space remaining at the front of the tweet.
  text = text.lstrip(' ') 
  # Remove @username
  text = re.sub('@[^\s]+','', text)
  text = list(text.translate(str.maketrans('', '', string.punctuation)).split())
  # correction of possible miss-click
  #text = ' '.join([str(tb(word).correct()) for word in text])
  text = ' '.join([str(word) for word in text])

  return text



pre_processing('it is a beautiful lif, https://github @matthew')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'beautiful lif'

In [37]:
data['text'] = data['text'].apply(pre_processing)

In [38]:
data

,text
0,films adapted comic books plenty success wheth...
1,every movie comes along suspect studio every i...
2,got mail works alot better deserves order make...
3,jaws rare film grabs attention shows single im...
4,moviemaking lot like general manager nfl team ...
...,...
1995,anything stigmata taken warning releasing simi...
1996,john boorman zardoz goofy cinematic debacle fu...
1997,kids hall acquired taste took least season wat...
1998,time john carpenter great horror director cour...


In [40]:
labels = [[1,0]] * (len(data['text'])//2) + [[0,1]] * (len(data['text'])//2)
len(labels)

2000

In [41]:
df = pd.DataFrame(labels, columns=['pos', 'neg'])

In [42]:
df

,pos,neg
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
1995,0,1
1996,0,1
1997,0,1
1998,0,1


In [43]:
complete_data = pd.concat([data, df], axis=1)

In [44]:
complete_data

,text,pos,neg
0,films adapted comic books plenty success wheth...,1,0
1,every movie comes along suspect studio every i...,1,0
2,got mail works alot better deserves order make...,1,0
3,jaws rare film grabs attention shows single im...,1,0
4,moviemaking lot like general manager nfl team ...,1,0
...,...,...,...
1995,anything stigmata taken warning releasing simi...,0,1
1996,john boorman zardoz goofy cinematic debacle fu...,0,1
1997,kids hall acquired taste took least season wat...,0,1
1998,time john carpenter great horror director cour...,0,1


This to show that $99\%$ of the sentences are under $947.14$ words.

In [54]:
import numpy as np
from nltk.tokenize import word_tokenize


seq_len = np.array([len(word_tokenize(sent)) for sent in complete_data['text']])
print([(p, np.percentile(seq_len, p)) for p in [65, 75, 80, 85, 90, 95, 99, 100]])

[(65, 418.0), (75, 463.25), (80, 494.20000000000005), (85, 535.0), (90, 608.1000000000001), (95, 720.05), (99, 947.1499999999999), (100, 1483.0)]


In [62]:
avg_length = seq_len.mean()
avg_length

386.225

[Bert Embedding](https://www.youtube.com/watch?v=zJW57aCBCTk)


[Fine-Tuning Bert](https://www.youtube.com/watch?v=x66kkDnbzi4)